In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import os
np.random.seed(1337)  # for reproducibility

import tensorflow as tf
from tensorflow.contrib.tpu.python.tpu import tpu_config
from tensorflow.contrib.tpu.python.tpu import tpu_estimator
from tensorflow.contrib.tpu.python.tpu import tpu_optimizer
import pdb

In [2]:

MODEL_DIR = './MLP/model_2'
DATA_DIR = './MLP_data'

prefetch_buffer_size = 128
num_files_infeed = 16
shuffle_buffer_size = 512
num_parallel_calls = 32

In [14]:
file_pattern = os.path.join(
        DATA_DIR, 'MLP_data_train*')

dataset = tf.data.Dataset.list_files(file_pattern)

In [15]:
dataset

<ShuffleDataset shapes: (), types: tf.string>

In [13]:
!ls ./MLP_data/MLP_data_train*

./MLP_data/MLP_data_train-1.tfrecords	 ./MLP_data/MLP_data_train-41.tfrecords
./MLP_data/MLP_data_train-10.tfrecords	 ./MLP_data/MLP_data_train-42.tfrecords
./MLP_data/MLP_data_train-100.tfrecords  ./MLP_data/MLP_data_train-43.tfrecords
./MLP_data/MLP_data_train-101.tfrecords  ./MLP_data/MLP_data_train-44.tfrecords
./MLP_data/MLP_data_train-102.tfrecords  ./MLP_data/MLP_data_train-45.tfrecords
./MLP_data/MLP_data_train-103.tfrecords  ./MLP_data/MLP_data_train-46.tfrecords
./MLP_data/MLP_data_train-104.tfrecords  ./MLP_data/MLP_data_train-47.tfrecords
./MLP_data/MLP_data_train-105.tfrecords  ./MLP_data/MLP_data_train-48.tfrecords
./MLP_data/MLP_data_train-106.tfrecords  ./MLP_data/MLP_data_train-49.tfrecords
./MLP_data/MLP_data_train-107.tfrecords  ./MLP_data/MLP_data_train-5.tfrecords
./MLP_data/MLP_data_train-108.tfrecords  ./MLP_data/MLP_data_train-50.tfrecords
./MLP_data/MLP_data_train-109.tfrecords  ./MLP_data/MLP_data_train-51.tfrecords
./MLP_data/MLP_data_train-11.tfrecords	 ./ML

In [10]:
!ls $file_pattern

./MLP_data/MLP_data_train-1.tfrecords	 ./MLP_data/MLP_data_train-41.tfrecords
./MLP_data/MLP_data_train-10.tfrecords	 ./MLP_data/MLP_data_train-42.tfrecords
./MLP_data/MLP_data_train-100.tfrecords  ./MLP_data/MLP_data_train-43.tfrecords
./MLP_data/MLP_data_train-101.tfrecords  ./MLP_data/MLP_data_train-44.tfrecords
./MLP_data/MLP_data_train-102.tfrecords  ./MLP_data/MLP_data_train-45.tfrecords
./MLP_data/MLP_data_train-103.tfrecords  ./MLP_data/MLP_data_train-46.tfrecords
./MLP_data/MLP_data_train-104.tfrecords  ./MLP_data/MLP_data_train-47.tfrecords
./MLP_data/MLP_data_train-105.tfrecords  ./MLP_data/MLP_data_train-48.tfrecords
./MLP_data/MLP_data_train-106.tfrecords  ./MLP_data/MLP_data_train-49.tfrecords
./MLP_data/MLP_data_train-107.tfrecords  ./MLP_data/MLP_data_train-5.tfrecords
./MLP_data/MLP_data_train-108.tfrecords  ./MLP_data/MLP_data_train-50.tfrecords
./MLP_data/MLP_data_train-109.tfrecords  ./MLP_data/MLP_data_train-51.tfrecords
./MLP_data/MLP_data_train-11.tfrecords	 ./ML

In [20]:
class MLP_Input(object):
  """Wrapper class that acts as the input_fn to TPUEstimator."""

  def __init__(self, is_training=True, is_eval=True, data_dir=None):
    self.is_eval = is_eval
    self.is_training = is_training
    self.data_dir = data_dir if data_dir else DATA_DIR

  def dataset_parser(self, value):
    """Parse an Imagenet record from value."""
    keys_to_features = {
        'X': tf.FixedLenFeature([], dtype=tf.string),
        'y': tf.FixedLenFeature(shape=[1], dtype=tf.int64)            
    }
    parsed = tf.parse_single_example(value, keys_to_features)
    X = tf.decode_raw(parsed['X'], tf.float32)
    X = tf.reshape(X, [10000])
    
    y = tf.cast(parsed['y'], tf.int64)
    return X, y

  def __call__(self, params):
    """Input function which provides a single batch for train or eval."""
    # Retrieves the batch size for the current shard. The # of shards is
    # computed according to the input pipeline deployment. See
    # `tf.contrib.tpu.RunConfig` for details.
    batch_size = params['batch_size']

    # Shuffle the filenames to ensure better randomization
    file_pattern = os.path.join(
        self.data_dir, 'MLP_data_train*' if self.is_training 
        else 'MLP_data_test*' )
    dataset = tf.data.Dataset.list_files(file_pattern)
    
    #pdb.set_trace()
    
    if self.is_training:
      dataset = dataset.shuffle(buffer_size=128)  # 1024 files in dataset

    if self.is_training:
      dataset = dataset.repeat()

    def prefetch_dataset(filename):
      buffer_size =  prefetch_buffer_size
      dataset = tf.data.TFRecordDataset(filename, buffer_size=buffer_size)
      return dataset

    dataset = dataset.apply(
        tf.contrib.data.parallel_interleave(
            prefetch_dataset, cycle_length= num_files_infeed,
            sloppy=True))
    dataset = dataset.shuffle(shuffle_buffer_size)

    dataset = dataset.map(
        self.dataset_parser,
        num_parallel_calls=num_parallel_calls)
    dataset = dataset.prefetch(batch_size)
    dataset = dataset.apply(
        tf.contrib.data.batch_and_drop_remainder(batch_size))

    dataset = dataset.prefetch(2)  # Prefetch overlaps in-feed with training
    images, labels = dataset.make_one_shot_iterator().get_next()
    
    if self.is_training or self.is_eval:
          return images, labels
    else:
          return dataset

In [21]:
my_data = MLP_Input(is_training=False)
X, y = my_data({'batch_size':10})

In [22]:
with tf.Session() as sess:
  valX, valy = sess.run([X, y])
  
    

In [23]:
valX.shape

(10, 10000)

In [25]:
valy

array([[1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [71]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
data_path = './MLP_data/MLP_data_test.tfrecords'  # address to save the hdf5 file
with tf.Session() as sess:
    feature =  {
        'X': tf.FixedLenFeature([], dtype=tf.string),
        'y': tf.FixedLenFeature(shape=[1], dtype=tf.int64)            
    }
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['X'], tf.float32)
    
    # Cast label data into int32
    label = tf.cast(features['y'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [10000])
    
    # Any preprocessing here ...
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=10, capacity=30, num_threads=1, min_after_dequeue=10)
    
        # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for batch_index in range(5):
        img, lbl = sess.run([images, labels])
        print(img.shape, lbl)
    # Stop the threads
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)
    sess.close()
    

(10, 10000) [[0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]]
(10, 10000) [[1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]]
(10, 10000) [[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]]
(10, 10000) [[1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]]
(10, 10000) [[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]]


In [72]:
img

array([[ 2.33810377,  1.39111209, -0.29905799, ..., -1.40061259,
        -0.09094267, -0.45051607],
       [-0.15256213,  0.07564683,  0.37871063, ..., -0.32956102,
         1.00066197, -0.14797863],
       [ 1.21993184,  1.79746187, -0.94660622, ...,  0.46586004,
        -0.00414874,  1.08658051],
       ..., 
       [ 1.32165027, -0.18541497,  1.36093473, ..., -1.13356423,
        -0.94628513,  0.81919318],
       [-0.6391685 , -1.20876503, -0.11585992, ...,  1.19735825,
        -1.26576638,  0.20602697],
       [-0.09409259,  0.17335817,  0.55533248, ..., -0.64466059,
         1.07462323,  0.84672332]], dtype=float32)